# Венгерский метод

In [3]:
import numpy as np


def check(costs, a, b):
    rows, cols = costs.shape
    allocation = np.zeros_like(costs)  # Матрица распределения
    supply = a.copy()
    demand = b.copy()
    print(f"Предложение:\n{supply}")
    print(f"Спрос:\n{demand}")
    zero_positions = [(i, j) for i in range(rows) for j in range(cols) if costs[i, j] == 0]

    for i, j in zero_positions:
        if supply[i] > 0 and demand[j] > 0:
            amount = min(supply[i], demand[j])  # Максимально возможное количество груза
            allocation[i, j] = amount
            supply[i] -= amount
            demand[j] -= amount

    print(f"Распределение перевозок:\n{allocation}")

    if sum(supply) == 0 and sum(demand) == 0:
        print("Можно выполнить перевозки по нулевым ячейкам.")
        return allocation, False
    else:
        print("Не удаётся полностью распределить перевозки по нулевым ячейкам.")
        print(f"Предложение:\n{supply}")
        print(f"Спрос:\n{demand}")
        return allocation, True



def zanulenie(costs):
    rows, cols = costs.shape

    for i in range(rows):
        min_row = min(costs[i])
        for j in range(cols):
            costs[i][j] -= min_row

    print(f"Вычитаем минимум по строкам\n{costs}")

    for j in range(cols):
        min_col = min(costs[i][j] for i in range(rows))
        for i in range(rows):
            costs[i][j] -= min_col

    print(f"Вычитаем минимум по столбцам\n{costs}")

def transformation(costs, allocation):
    rows, cols = costs.shape
    import numpy as np

    zero_column = None
    for j in range(cols):
        if np.all(allocation[:, j] == 0):
            zero_column = j
            break

    if zero_column is None:
        for j in range(cols):
            zero_count = np.sum(costs[:, j] == 0)
            if zero_count == 1:
                zero_column = j
                break


    print(f"Найден столбец: {zero_column}")

    min_val = min(costs[i, zero_column] for i in range(rows) if costs[i, zero_column] > 0)
    print(f"Минимальное ненулевое значение в costs в столбце {zero_column}: {min_val}")

    for i in range(rows):
        costs[i, zero_column] -= min_val

    for i in range(rows):
        if costs[i, zero_column] < 0:
            costs[i, :] += min_val
            break

    print(f"Обновлённая матрица costs:\n{costs}")
    return costs


def main():
    a = [1600, 1000, 1700]
    b = [1000, 1700, 1600]
    cost = [
        [4883, 4280, 6213],
        [5327, 4296, 6188],
        [6006, 5030, 7224]
    ]
    costs = np.array(cost)
    flag = True
    i = 0
    allocation = np.zeros(costs.shape)
    while flag:
        print(f"Итерация номер {i}-------------------------------------")
        zanulenie(costs)
        allocation, flag = check(costs, a, b)
        if(flag):
            transformation(costs, allocation)
        i+=1
    total_cost = np.sum(cost * allocation)
    print(f"Цена после оптимизации {total_cost}")


if __name__ == "__main__":
    main()

Итерация номер 0-------------------------------------
Вычитаем минимум по строкам
[[ 603    0 1933]
 [1031    0 1892]
 [ 976    0 2194]]
Вычитаем минимум по столбцам
[[  0   0  41]
 [428   0   0]
 [373   0 302]]
Предложение:
[1600, 1000, 1700]
Спрос:
[1000, 1700, 1600]
Распределение перевозок:
[[1000  600    0]
 [   0 1000    0]
 [   0  100    0]]
Не удаётся полностью распределить перевозки по нулевым ячейкам.
Предложение:
[0, 0, 1600]
Спрос:
[0, 0, 1600]
Найден столбец: 2
Минимальное ненулевое значение в costs в столбце 2: 41
Обновлённая матрица costs:
[[  0   0   0]
 [469  41   0]
 [373   0 261]]
Итерация номер 1-------------------------------------
Вычитаем минимум по строкам
[[  0   0   0]
 [469  41   0]
 [373   0 261]]
Вычитаем минимум по столбцам
[[  0   0   0]
 [469  41   0]
 [373   0 261]]
Предложение:
[1600, 1000, 1700]
Спрос:
[1000, 1700, 1600]
Распределение перевозок:
[[1000  600    0]
 [   0    0 1000]
 [   0 1100    0]]
Не удаётся полностью распределить перевозки по нулевы

# Метод потенциалов

In [11]:
import numpy as np

#Метод с-з угла
def northwest_corner_method(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    rows, cols = len(supply), len(demand)
    allocation = np.zeros((rows, cols), dtype=int)

    i, j = 0, 0
    while i < rows and j < cols:
        allocated = min(supply[i], demand[j])
        allocation[i, j] = allocated
        supply[i] -= allocated
        demand[j] -= allocated

        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1

    return allocation

# Подсчитываем общую стоимость перевозок
def calculate_cost(cost_matrix, allocation):
    total_cost = np.sum(cost_matrix * allocation)
    return total_cost

def potentials_method(cost_matrix, allocation):
    rows, cols = cost_matrix.shape

    u = np.full(rows, None)
    v = np.full(cols, None)
    optimal_matrix = np.full((rows, cols), 0)
    u[0] = 0
    u, v, optimal_matrix = calculate_potentials(allocation, cost_matrix)# Оценка стоимости для небазисных
    delta = np.full((rows, cols), 0)
    for i in range(rows):
        for j in range(cols):
            if allocation[i, j] == 0:
                # Считаем матрицу потенциалов
                optimal_matrix[i, j] = cost_matrix[i, j] - (u[i] + v[j])
                delta[i, j] = (u[i] + v[j]) - cost_matrix[i, j]
    print(f"U{u}")
    print(f"V{v}")
    print(f"Матрица для проверки\n{optimal_matrix}")

    if np.all(optimal_matrix >= 0):
        return allocation, calculate_cost(cost_matrix, allocation)

    while np.any(optimal_matrix < 0):
            allocation = optimize_allocation(allocation, delta)
            print(allocation)
            u, v, optimal_matrix = calculate_potentials(allocation, cost_matrix)
            print(f"U{u}")
            print(f"V{v}")
            print(f"Матрица для проверки\n{optimal_matrix}")
            delta = calculate_delta(cost_matrix, u, v)

    return allocation, calculate_cost(cost_matrix, allocation)

def find_cycle(allocation, start):
    """Найти цикл для оптимизации плана."""
    rows, cols = allocation.shape
    visited = set()
    path = []

    def dfs(node, direction):
        if node in visited:
            # Если цикл найден, возвращаем его
            if node == start:
                path.append(node)
                return True
            return False

        visited.add(node)
        path.append(node)
        i, j = node

        if direction == "row":
            for col in range(cols):
                if (allocation[i, col] > 0 and (i, col) != node) or (i, col) == start:
                    if dfs((i, col), "col"):
                        return True
        elif direction == "col":
            for row in range(rows):
                if (allocation[row, j] > 0 and (row, j) != node) or (row, j) == start:
                    if dfs((row, j), "row"):
                        return True

        # Удаляем узел, если он не часть цикла
        path.pop()
        return False

    # Запускаем поиск цикла
    found = dfs(start, "row")

    # Если путь замкнулся (начальная точка совпадает с конечной), возвращаем найденный цикл.
    if found and path[0] == path[-1]:
        path.pop()
        print(f"Путь{path}")
        return path
    return None



def optimize_allocation(allocation, delta):
    i, j = np.unravel_index(np.argmax(delta), delta.shape)
    cycle = find_cycle(allocation, (i, j))
    if not cycle:
        raise ValueError("Цикл не найден, что-то пошло не так.")

    # Чередование знаков: '+' для увеличения, '-' для уменьшения
    signs = [1 if k % 2 == 0 else -1 for k in range(len(cycle))]
    values = [allocation[x, y] for (x, y), sign in zip(cycle, signs) if sign == -1]
    theta = min(values)  # Минимум из клеток с '-' знаком
    # Обновление плана
    for (x, y), sign in zip(cycle, signs):
        allocation[x, y] += sign * theta

    return allocation


def calculate_potentials(allocation, cost_matrix):
    rows, cols = cost_matrix.shape

    u = np.full(rows, None)
    v = np.full(cols, None)
    u[0] = 0  # Начинаем с первого элемента

    # Основной цикл для вычисления потенциалов
    while None in u or None in v:
        progress = False  # Флаг для отслеживания изменений
        for i in range(rows):
            for j in range(cols):
                if allocation[i, j] > 0:  # Базисная клетка
                    if u[i] is not None and v[j] is None:
                        v[j] = cost_matrix[i, j] - u[i]
                        progress = True
                    elif v[j] is not None and u[i] is None:
                        u[i] = cost_matrix[i, j] - v[j]
                        progress = True


        # Если потенциалы не обновились
        if not progress:
            min_cost, min_posit = find_min_cost_with_none(u, v, cost_matrix)
            if u[min_posit[0]] is not None and v[min_posit[1]] is None:
                v[min_posit[1]] = cost_matrix[min_posit[0], min_posit[1]] - u[min_posit[0]]
            elif u[min_posit[0]] is None and v[min_posit[1]] is not None:
                u[min_posit[0]] = cost_matrix[min_posit[0], min_posit[1]] - v[min_posit[1]]
            else: u[min_posit[0]] = 0

    # Оценка стоимости для небазисных клеток
    optimal_matrix = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            if allocation[i, j] == 0:
                optimal_matrix[i, j] = cost_matrix[i, j] - (u[i] + v[j])

    return u, v, optimal_matrix


def find_min_cost_with_none(u, v, cost_matrix):
    rows, cols = cost_matrix.shape
    min_cost = float('inf')
    min_cost_position = (-1, -1)

    # Ищем минимальную стоимость среди клеток, где u или v равны None
    for i in range(rows):
        for j in range(cols):
            if u[i] is None or v[j] is None:
                if cost_matrix[i, j] < min_cost:
                    min_cost = cost_matrix[i, j]
                    min_cost_position = (i, j)

    return min_cost, min_cost_position

def calculate_delta(cost_matrix, u, v):
    rows, cols = cost_matrix.shape

    delta = np.full((rows, cols), None)
    for i in range(rows):
        for j in range(cols):
            delta[i, j] = (u[i] + v[j]) - cost_matrix[i, j]
    return delta

supply = [1600, 1000, 1700] #Запасы
demand = [1000, 1700, 1600] # Потребности
cost_matrix = np.array([
    [4883, 4280, 6213],
    [5327, 4296, 6198],
    [6006, 5030, 7224]
])

initial_allocation = northwest_corner_method(supply, demand)
print("Начальный опорный план:")
print(initial_allocation)

# Шаг 2: Стоимость начального плана
initial_cost = calculate_cost(cost_matrix, initial_allocation)
print(f"Стоимость начального плана: {initial_cost}")
#
# # Шаг 3: Метод потенциалов для оптимизации
optimal_allocation, optimal_cost = potentials_method(cost_matrix, initial_allocation)
print("Оптимальный план:")
print(optimal_allocation)
print(f"Минимальная стоимость перевозки: {optimal_cost}")

Начальный опорный план:
[[1000  600    0]
 [   0 1000    0]
 [   0  100 1600]]
Стоимость начального плана: 23808400
U[0 16 750]
V[4883 4280 6474]
Матрица для проверки
[[   0.    0. -261.]
 [ 428.    0. -292.]
 [ 373.    0.    0.]]
Путь[(1, 2), (1, 1), (2, 1), (2, 2)]
[[1000  600    0]
 [   0    0 1000]
 [   0 1100  600]]
U[0 -276 750]
V[4883 4280 6474]
Матрица для проверки
[[   0.    0. -261.]
 [ 720.  292.    0.]
 [ 373.    0.    0.]]
Путь[(0, 2), (0, 1), (2, 1), (2, 2)]
[[1000    0  600]
 [   0    0 1000]
 [   0 1700    0]]
U[0 -15 750]
V[4883 4280 6213]
Матрица для проверки
[[  0.   0.   0.]
 [459.  31.   0.]
 [373.   0. 261.]]
Оптимальный план:
[[1000    0  600]
 [   0    0 1000]
 [   0 1700    0]]
Минимальная стоимость перевозки: 23359800
